In [1101]:

## based on 
# https://danijar.com/building-variational-auto-encoders-in-tensorflow/
# https://towardsdatascience.com/teaching-a-variational-autoencoder-vae-to-draw-mnist-characters-978675c95776
## Imports

import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn.model_selection
import os, glob, shutil, math, time
# reset tf graph  
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)
#sess = tf.Session()
session = tf.InteractiveSession()
## data pipeline from R-prepared CSV

df = pd.read_csv("split_x.csv",sep=",")
data = df.values
odf = pd.read_csv("split_y.csv",sep=",")
outcomes = odf.values


## seed 
np.random.seed(134)

m = np.shape(data)[0]
d = np.shape(data)[1]
p = 1

## Functions
def data_rescale(scaled_dat,train_mean,train_var):
    d = np.shape(train_mean)[0]
#     print('unnormalizing with number of dimensions = ' +str(d))
    dat = (np.multiply(scaled_dat.T,np.sqrt(train_var),)+train_mean).T
    return(dat)

def data_normalize(data,train_mean,train_var):
    d = np.shape(train_mean)[0]
    print('normalizing with number of dimensions = ' +str(d))
    scaled_dat = np.divide((data.T - train_mean),np.sqrt(train_var),).T
    return(scaled_dat)
    
## data generation
# get random partition
msk = np.random.rand(np.shape(data)[0]) < 0.70
print('msk[8] = ' + str(msk[8]))

# test and train split
train_data_x = data[msk]
test_data_x = data[~msk]
train_data_y = outcomes[msk]
test_data_y = outcomes[~msk]



# non-dimenisionalization
train_var_x = np.var(train_data_x,0).reshape(d,1)
train_mean_x = np.mean(train_data_x,0).reshape(d,1)
scaled_train_data_x = data_normalize(train_data_x,train_mean_x,train_var_x)
scaled_test_data_x = data_normalize(test_data_x,train_mean_x,train_var_x)

train_var_y = np.var(train_data_y,0).reshape(p,1)
train_mean_y = np.mean(train_data_y,0).reshape(p,1)
scaled_train_data_y = data_normalize(train_data_y,train_mean_y,train_var_y)
scaled_test_data_y = data_normalize(test_data_y,train_mean_y,train_var_y)

# scaled_train_data_x = np.array([[1,2,3,1],[2,3,4,1],[6,1,2,1]])
# scaled_train_data_y =tf.reshape(np.array([0.5,0.58,0.77]),shape=[3,1])
# scaled_test_data_x = np.array([[9,7,8,2],[8,2,2,1]])
# scaled_test_data_y = tf.reshape(np.array([0.55,0.8]),shape=[2,1])
# d = 3
n_input = d # data input (img shape: 28*28)
n =  np.shape(scaled_train_data_x)[0]
ntest =  np.shape(scaled_test_data_x)[0]

## tf variable placeholders for feeding
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32,[None,n_input],name="X")
    Y = tf.placeholder(tf.float32,[None,n_input],name="Y")
    global_step = tf.Variable(0, trainable=False)
    keep_prob = tf.placeholder(dtype=tf.float32, shape=(), name='keep_prob')

print(scaled_train_data_x.shape)

msk[8] = True
normalizing with number of dimensions = 155
normalizing with number of dimensions = 155
normalizing with number of dimensions = 1
normalizing with number of dimensions = 1
(983, 155)


In [1102]:
def myPairwiseDist(X,Y):
    tile1 = tf.tile(tf.expand_dims(X, 0), [Y.shape[0], 1, 1])
    tile2 = tf.tile(tf.expand_dims(Y, 1), [1,X.shape[0] , 1])
    pairwiseEuclideanDistance = tf.reduce_sum(tf.square(tf.subtract(tile1, tile2)), 2)
    return pairwiseEuclideanDistance
def myRBFKernel(gamma,X,Y):
    K = tf.exp(tf.multiply(tf.cast(-1*gamma,dtype="float64"), tf.cast(myPairwiseDist(X,Y),dtype="float64")))
    return K
def kernelModel(alpha,K):
    ymod = tf.matmul(K,alpha)
    return ymod
def ridgeReg(alpha):
    penalty = tf.norm(alpha)
    return penalty

def objective(alpha,ytrain,K,lam):
    o = tf.cast(tf.nn.l2_loss(tf.subtract(ytrain,kernelModel(alpha,K))),dtype="float64") + tf.cast(tf.abs(lam),dtype="float64")*tf.cast(ridgeReg(alpha),dtype="float64")
    return o
    
def predictiveModel(x,alpha,xtrain,gamma):
    predK = kernelModel(alpha,myRBFKernel(xtrain,x))
    return predK
def solveLinearKernel(K,lam,ytrain):
    mat = tf.add(tf.matmul(tf.transpose(K),K),lam*tf.eye(num_rows=tf.cast(scaled_train_data_y.shape[0],dtype="int32"),
                                                         dtype="float64"))
    rhs = tf.matmul(tf.transpose(K),ytrain)
    alpha = tf.matrix_solve(mat,rhs)
    return alpha

In [1103]:
#3 intial weights
alpha = tf.Variable(tf.random_normal([n,1],mean=0.0,stddev=1.0,dtype="float64"),name="alpha", dtype="float64")

In [1104]:
## set kernel
gamma = tf.Variable(1.930698e-02, tf.float64)
K  = myRBFKernel(gamma,scaled_train_data_x,scaled_train_data_x)
Kt  = myRBFKernel(gamma,scaled_train_data_x,scaled_test_data_x)

In [1105]:
## linear solve
lam = tf.Variable(1.668101e-06, tf.float64)
bestAlpha = solveLinearKernel(K,1.668101e-06,scaled_train_data_y)


In [1106]:
## define performance metrics
train_pred = kernelModel(alpha,K)
test_pred = kernelModel(alpha,Kt)
train_errors = tf.subtract(train_pred,scaled_train_data_y)
test_errors = tf.subtract(test_pred,scaled_test_data_y)
train_mae = tf.metrics.mean_absolute_error(train_pred,scaled_train_data_y)
test_mae = tf.metrics.mean_absolute_error(test_pred,scaled_test_data_y)
train_rmse = tf.metrics.mean_squared_error(train_pred*np.sqrt(train_var_y),scaled_train_data_y*np.sqrt(train_var_y))
test_rmse = tf.metrics.mean_squared_error(test_pred*np.sqrt(train_var_y),scaled_test_data_y*np.sqrt(train_var_y))


## linear solve
train_pred_ds = kernelModel(bestAlpha,K)
test_pred_ds = kernelModel(bestAlpha,Kt)
train_errors_ds = tf.subtract(train_pred_ds,scaled_train_data_y)
test_errors_ds = tf.subtract(test_pred_ds,scaled_test_data_y)
train_mae_ds = tf.metrics.mean_absolute_error(train_pred_ds,scaled_train_data_y)
test_mae_ds = tf.metrics.mean_absolute_error(test_pred_ds,scaled_test_data_y)
train_rmse_ds = tf.metrics.mean_squared_error(train_pred_ds*np.sqrt(train_var_y),scaled_train_data_y*np.sqrt(train_var_y))
test_rmse_ds = tf.metrics.mean_squared_error(test_pred_ds*np.sqrt(train_var_y),scaled_test_data_y*np.sqrt(train_var_y))



In [1107]:
## build optimizer
optimizer =tf.train.AdamOptimizer(learning_rate=1)
loss = objective(alpha,scaled_train_data_y,K,lam)
rt = optimizer.minimize(loss,var_list=[alpha])


In [1108]:
## initialize all
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
session.run(init)
session.run(init_l)


In [1109]:
alpha_err = tf.reduce_sum(tf.abs(tf.subtract(alpha,bestAlpha)))
print("starting at", "loss:", session.run(loss),'err',session.run(alpha_err),'gamma',session.run(gamma))


starting at loss: 13572.969553468789 err 18187.173681824635 gamma 0.01930698


In [1110]:
# session.close()

In [1115]:
for step in range(1500):  
  session.run(rt)
  if not step % 100:
#       print("step", step, "loss:", session.run(loss),'err',session.run(alpha_err))
        print("step", step, "loss:", session.run(loss),'gamma',session.run(gamma),'lambda',session.run(lam))
#print("x:", session.run(alpha))
#print('best x',session.run(bestAlpha))        

step 0 loss: 64.10745350287264 gamma 0.01930698 lambda 1.668101e-06
step 100 loss: 49.860938387185065 gamma 0.01930698 lambda 1.668101e-06
step 200 loss: 47.253932326200655 gamma 0.01930698 lambda 1.668101e-06
step 300 loss: 2258.261953474609 gamma 0.01930698 lambda 1.668101e-06
step 400 loss: 232.0811268992046 gamma 0.01930698 lambda 1.668101e-06
step 500 loss: 12572.382809064477 gamma 0.01930698 lambda 1.668101e-06
step 600 loss: 39.56998783084887 gamma 0.01930698 lambda 1.668101e-06
step 700 loss: 170.21020646795384 gamma 0.01930698 lambda 1.668101e-06
step 800 loss: 37.18586976559926 gamma 0.01930698 lambda 1.668101e-06
step 900 loss: 1089.085788523047 gamma 0.01930698 lambda 1.668101e-06
step 1000 loss: 35.45129074116178 gamma 0.01930698 lambda 1.668101e-06
step 1100 loss: 34.377968629065705 gamma 0.01930698 lambda 1.668101e-06
step 1200 loss: 37.04460889947174 gamma 0.01930698 lambda 1.668101e-06
step 1300 loss: 33.11816187716755 gamma 0.01930698 lambda 1.668101e-06
step 1400 los

In [1116]:
print('train op done, metrics' )
print('MAEs',[session.run(train_mae)[1],session.run(test_mae)[1]])
print('RMSE',[session.run(train_rmse)[1],session.run(test_rmse)[1]])

print('train op done, metrics, units' )
resc =  lambda x: str(x*np.sqrt(train_var_y)).strip('[]')
print('MAEs',resc(session.run(train_mae)[1]),"|",resc(session.run(test_mae)[1]))
print('RMSE',np.sqrt(session.run(train_rmse)[1]),"|",np.sqrt(session.run(test_rmse)[1]))


train op done, metrics
MAEs [0.22098398, 0.27203837]
RMSE [71.5568, 101.97008]
train op done, metrics, units
MAEs 5.51448304 | 6.85559465
RMSE 8.0526905 | 9.7313595


In [1114]:

print('train op done, metrics, units, linear solve' )
resc =  lambda x: str(x*np.sqrt(train_var_y)).strip('[]')
print('MAEs',resc(session.run(train_mae_ds)[1]),"|",resc(session.run(test_mae_ds)[1]))
print('RMSE',np.sqrt(session.run(train_rmse_ds)[1]),"|",np.sqrt(session.run(test_rmse_ds)[1]))

train op done, metrics, units, linear solve
MAEs 0.67358796 | 3.31031016
RMSE 1.369667 | 6.523318


In [974]:
session.close()

In [730]:
np.sqrt((6.25909194**2))

6.25909194

In [942]:
np.sqrt(train_var_y)

array([[25.70649411]])

In [1100]:
np.sqrt(np.mean(np.power(test_errors.eval()*np.sqrt(train_var_y),2)))

15.875613548755723